In [2]:
!pip install neurokit2 scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.2/689.2 kB 3.9 MB/s eta 0:00:00


In [16]:
import neurokit2 as nk
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from google.colab import drive
from scipy.io import loadmat

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, r2_score
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Load the dataset from Google Drive
file_path = '/content/drive/MyDrive/ppg file/Sample_PPG_MAT_125Hz.mat'

# Load dataset
data = loadmat(file_path)
ppg_signal = data['Data'].flatten()
print(data.keys())
print(data['Fs'].flatten())

# Preprocess the PPG signal
ppg_cleaned = nk.ppg_clean(ppg_signal, sampling_rate=125)

# Extract PPG signals (including heart rate)
signals, info = nk.ppg_process(ppg_cleaned, sampling_rate=125)
print(signals.columns)

# Extract heart rate and additional features
heart_rate = signals['PPG_Rate']


# Calculate Inter-Beat Interval (IBI) from peaks
peaks = signals['PPG_Peaks']
IBI = np.diff(np.where(peaks == 1)[0]) / 125.0  # Convert sample indices to time in seconds

# Calculate Heart Rate (HR)
average_IBI = np.mean(IBI)
calculated_heart_rate = 60 / average_IBI  # Heart rate in BPM
print(f"Calculated Heart Rate (HR): {calculated_heart_rate}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dict_keys(['__header__', '__version__', '__globals__', 'Data', 'Fs', 'source'])
[125]
Index(['PPG_Raw', 'PPG_Clean', 'PPG_Rate', 'PPG_Quality', 'PPG_Peaks'], dtype='object')
Calculated Heart Rate (HR): 99.22766957689727


In [14]:
# Extract additional features for training
features = signals[['PPG_Rate', 'PPG_Clean', 'PPG_Quality', 'PPG_Peaks']]


# Extract available features for training
# Exclude PPG_Rate from features as it is used as ground truth
selected_features = [col for col in signals.columns if col != 'PPG_Rate']
features = signals[selected_features]

# Combine features and ground truth heart rate into a dataset
dataset = pd.DataFrame(features)
dataset['Ground_Truth_HR'] = ground_truth_heart_rate

# Assuming you have ground truth heart rate, replace with your ground truth data if available
# ground_truth_heart_rate = heart_rate #

# Combine features and ground truth heart rate into a dataset
# dataset = features.copy()
# dataset['Ground_Truth_HR'] = ground_truth_heart_rate #

# Split the dataset into training and testing sets
X = dataset.drop(columns=['Ground_Truth_HR'])
y = dataset['Ground_Truth_HR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a random forest model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=300, random_state=48)
model.fit(X_train, y_train)

# Predict heart rate on the test set
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")


Mean Absolute Error: 1.3917362049493767
R2 Score: 0.1930486612542348
